In [1]:
import numpy as np
import pandas as pd
from alpharaw import register_all_readers
from alpharaw.ms_data_base import ms_reader_provider

register_all_readers()
hdf5_reader = ms_reader_provider.get_reader("hdf5")
hdf5_reader.load_raw("./output/20200317_QE_HFX2_LC3_DIA_RA957_R01.mzML.hdf5")

/Users/zshang/Projects/DIA-Aspire-rescore/.venv/lib/python3.13/site-packages/alpharaw/raw_access/pysciexwifffilereader.py:44: UserWarning: Dotnet-based dependencies could not be loaded. Sciex support is disabled.
  warnings.warn(
/Users/zshang/Projects/DIA-Aspire-rescore/.venv/lib/python3.13/site-packages/alpharaw/raw_access/pythermorawfilereader.py:35: UserWarning: Dotnet-based dependencies could not be loaded. Thermo support is disabled.
  warnings.warn(


# only run once
mzml_reader = ms_reader_provider.get_reader("mzml")
raw_data = mzml_reader.load_raw("../data/raw/SYS026_RA957/20200317_QE_HFX2_LC3_DIA_RA957_R01.mzML")
mzml_reader.save_hdf("./output/20200317_QE_HFX2_LC3_DIA_RA957_R01.mzML.hdf5")

mzml_reader = ms_reader_provider.get_reader("mzml")
raw_data = mzml_reader.load_raw("../data/raw/SYS026_RA957/20200317_QE_HFX2_LC3_DIA_RA957_R02.mzML")
mzml_reader.save_hdf("./output/20200317_QE_HFX2_LC3_DIA_RA957_R02.mzML.hdf5")

In [12]:
spec_df = hdf5_reader.spectrum_df
peak_df = hdf5_reader.peak_df
spec_df

,isolation_lower_mz,isolation_upper_mz,ms_level,peak_start_idx,peak_stop_idx,precursor_charge,precursor_mz,rt,spec_idx
0,-1.0,-1.0,1,0,3283,0,-1.0,0.002374,0
1,350.0,387.0,2,3283,4654,0,368.5,0.006591,1
2,386.0,416.0,2,4654,5532,0,401.0,0.007747,2
3,415.0,439.0,2,5532,6317,0,427.0,0.008868,3
4,438.0,462.0,2,6317,7198,0,450.0,0.009991,4
...,...,...,...,...,...,...,...,...,...
89076,883.0,955.0,2,137080673,137082379,0,919.0,119.995220,89076
89077,954.0,1057.0,2,137082379,137084652,0,1005.5,119.996410,89077
89078,1056.0,1650.0,2,137084652,137086933,0,1353.0,119.998780,89078
89079,-1.0,-1.0,1,137086933,137092701,0,-1.0,120.000600,89079


In [13]:
# check the spectra number between two adjacent ms1
ms1_idxes = spec_df[spec_df['ms_level'] == 1].index.to_numpy()
results = []    
for i in range(len(ms1_idxes) - 1):
    start_idx = ms1_idxes[i] + 1
    end_idx = ms1_idxes[i + 1]
    num_ms2 = end_idx - start_idx
    results.append((ms1_idxes[i], ms1_idxes[i + 1], num_ms2))

summary = pd.DataFrame(results, columns=['ms1_start', 'ms1_end', 'num_ms2'])
summary.describe()

,ms1_start,ms1_end,num_ms2
count,3873.000000,3873.000000,3873.0
mean,44528.000000,44551.000000,22.0
std,25718.211864,25718.211864,0.0
min,0.000000,23.000000,22.0
25%,22264.000000,22287.000000,22.0
50%,44528.000000,44551.000000,22.0
75%,66792.000000,66815.000000,22.0
max,89056.000000,89079.000000,22.0


In [14]:
def get_cycle_time(spectrum_df):
    ms1_mask = spectrum_df["ms_level"] == 1
    ms1_rts = spectrum_df.loc[ms1_mask, "rt"].values

    if len(ms1_rts) < 2:
        return np.array([])

    return np.diff(ms1_rts)


def get_scan_times(spectrum_df):
    rts = spectrum_df["rt"].values
    return np.diff(rts)


cycle_times = get_cycle_time(spec_df)
scan_times = get_scan_times(spec_df)

In [15]:
cycle_times.mean(), cycle_times.std(), cycle_times.min(), cycle_times.max()

(np.float64(0.030983275613916863),
 np.float64(0.0002627860506663159),
 np.float64(0.030585000000002083),
 np.float64(0.03148969999999984))

In [16]:
scan_times.mean(), scan_times.std(), scan_times.min(), scan_times.max()

(np.float64(0.0013471311905332284),
 np.float64(0.0006642162628914645),
 np.float64(0.0011060000000071568),
 np.float64(0.004239999999995803))

In [17]:
df_diann = pd.read_parquet("../data/raw/SYS026_RA957/DDA_SYSMHC_bynam/lib-base-result-first-pass.parquet")
df_diann = df_diann[df_diann["Run"] == "20200317_QE_HFX2_LC3_DIA_RA957_R01"]

In [18]:
spec_df["rt"].dtypes, df_diann["RT"].dtypes

(dtype('float64'), dtype('float32'))

In [19]:
# transform rt from float64 to float32
spec_df["rt"] = spec_df["rt"].astype(np.float32)

# try to fetch a spectrum by rt
target_rt = df_diann["RT"].iloc[0]
spec_df[spec_df["rt"] == target_rt]

,isolation_lower_mz,isolation_upper_mz,ms_level,peak_start_idx,peak_stop_idx,precursor_charge,precursor_mz,rt,spec_idx
67077,547.0,568.0,2,99193934,99195777,0,557.5,90.442459,67077


In [20]:
# try every row in df_diann
# to see if we can find matching rt in spec_df
# if cannot, find the closest one and save them in np.array

matched_count = 0
closest_rts = []
rt_differences = []
for rt in df_diann["RT"]:
    if not spec_df[spec_df["rt"] == rt].empty:
        matched_count += 1
    else:
        # find closest rt, and its difference
        closest_rt = spec_df.iloc[(spec_df["rt"] - rt).abs().argsort()[:1]]["rt"].values[0]
        closest_rts.append(closest_rt)
        rt_differences.append(abs(closest_rt - rt))

matched_count, len(df_diann), matched_count / len(df_diann)

(159314, 169226, 0.941427440227861)

In [21]:
min(rt_differences), max(rt_differences), np.mean(rt_differences), np.std(rt_differences)

(np.float32(5.9604645e-08),
 np.float32(7.6293945e-06),
 np.float32(4.9382015e-06),
 np.float32(2.3214732e-06))